In [1]:
# general imports
import numpy as np
import math
import matplotlib.pyplot as plt
# magic word for producing visualizations in notebook
%matplotlib inline
from braket.circuits import Circuit, circuit
from braket.devices import LocalSimulator
from braket.aws import AwsSession, AwsDevice

In [2]:
def get_unitary(self):
    """
    Function to get the unitary matrix corresponding to an entire circuit.
    Acts on self and returns the corresponding unitary
    """
    num_qubits = int(
        max(self.qubits) + 1
    )  # Coincides with self.qubit_count when qubit indexing is contiguous.

    # Define the unitary matrix. Start with the identity matrix.
    # Reshape the unitary into a tensor with the right number of indices (given by num_qubits)
    unitary = np.reshape(np.eye(2 ** num_qubits, 2 ** num_qubits), [2] * 2 * num_qubits)

    # Iterate over the moments in the circuit
    for key in self.moments:

        # Get the matrix corresponding to the gate
        matrix = self.moments[key].operator.to_matrix()
        # Get the target indices for the gate
        targets = self.moments[key].target

        # Reshape the gate matrix
        gate_matrix = np.reshape(matrix, [2] * len(targets) * 2)

        # Construct a tuple specifying the axes along which we contract (i.e., which qubits the gate acts on)
        axes = (
            np.arange(len(targets), 2 * len(targets)),
            targets,
        )

        # Apply the gate by contracting the existing unitary with the new gate
        unitary = np.tensordot(gate_matrix, unitary, axes=axes)

        # tensordot causes the axes contracted to end up in the first positions.
        # We'll need to invert this permutation to put the indices in the correct place

        # Find the indices that are not used
        unused_idxs = [idx for idx in range(2 * num_qubits) if idx not in targets]

        # The new order of indices is given by
        permutation = list(targets) + unused_idxs

        # Find the permutation that undoes this reordering
        inverse_permutation = np.argsort(permutation)

        # Relabel the qubits according to this inverse_permutation
        unitary = np.transpose(unitary, inverse_permutation)

    # Reshape to a 2^N x 2^N matrix (for N=num_qubits)and return
    unitary = np.reshape(unitary, (2 ** num_qubits, 2 ** num_qubits))
    return unitary


def adjoint(self):
    """Generates a circuit object corresponding to the adjoint of a given circuit, in which the order
    of gates is reversed, and each gate is the adjoint (i.e., conjugate transpose) of the original.
    """

    adjoint_circ = Circuit()

    # Loop through the instructions (gates) in the circuit:
    for instruction in self.instructions:
        # Save the operator name and target
        op_name = instruction.operator.name
        target = instruction.target
        angle = None
        # If the operator has an attribute called 'angle', save that too
        if hasattr(instruction.operator, "angle"):
            angle = instruction.operator.angle

        # To make use of native gates, we'll define the adjoint for each
        if op_name == "H":
            adjoint_gate = Circuit().h(target)
        elif op_name == "I":
            adjoint_gate = Circuit().i(target)
        elif op_name == "X":
            adjoint_gate = Circuit().x(target)
        elif op_name == "Y":
            adjoint_gate = Circuit().y(target)
        elif op_name == "Z":
            adjoint_gate = Circuit().z(target)
        elif op_name == "S":
            adjoint_gate = Circuit().si(target)
        elif op_name == "Si":
            adjoint_gate = Circuit().s(target)
        elif op_name == "T":
            adjoint_gate = Circuit().ti(target)
        elif op_name == "Ti":
            adjoint_gate = Circuit().t(target)
        elif op_name == "V":
            adjoint_gate = Circuit().vi(target)
        elif op_name == "Vi":
            adjoint_gate = Circuit().v(target)
        elif op_name == "Rx":
            adjoint_gate = Circuit().rx(target, -angle)
        elif op_name == "Ry":
            adjoint_gate = Circuit().ry(target, -angle)
        elif op_name == "Rz":
            adjoint_gate = Circuit().rz(target, -angle)
        elif op_name == "PhaseShift":
            adjoint_gate = Circuit().phaseshift(target, -angle)
        elif op_name == "CNot":
            adjoint_gate = Circuit().cnot(*target)
        elif op_name == "Swap":
            adjoint_gate = Circuit().swap(*target)
        elif op_name == "ISwap":
            adjoint_gate = Circuit().pswap(*target, -np.pi / 2)
        elif op_name == "PSwap":
            adjoint_gate = Circuit().pswap(*target, -angle)
        elif op_name == "XY":
            adjoint_gate = Circuit().xy(*target, -angle)
        elif op_name == "CPhaseShift":
            adjoint_gate = Circuit().cphaseshift(*target, -angle)
        elif op_name == "CPhaseShift00":
            adjoint_gate = Circuit().cphaseshift00(*target, -angle)
        elif op_name == "CPhaseShift01":
            adjoint_gate = Circuit().cphaseshift01(*target, -angle)
        elif op_name == "CPhaseShift10":
            adjoint_gate = Circuit().cphaseshift10(*target, -angle)
        elif op_name == "CY":
            adjoint_gate = Circuit().cy(*target)
        elif op_name == "CZ":
            adjoint_gate = Circuit().cz(*target)
        elif op_name == "XX":
            adjoint_gate = Circuit().xx(*target, -angle)
        elif op_name == "YY":
            adjoint_gate = Circuit().yy(*target, -angle)
        elif op_name == "ZZ":
            adjoint_gate = Circuit().zz(*target, -angle)
        elif op_name == "CCNot":
            adjoint_gate = Circuit().ccnot(*target)
        elif op_name == "CSwap":
            adjoint_gate = Circuit().cswap(*target)

        # If the gate is a custom unitary, we'll create a new custom unitary
        else:
            # Extract the transpose of the unitary matrix for the unitary gate
            adjoint_matrix = instruction.operator.to_matrix().T.conj()

            # Define a gate for which the unitary matrix is the adjoint found above.
            # Add an "H" to the display name.
            adjoint_gate = Circuit().unitary(
                matrix=adjoint_matrix,
                targets=instruction.target,
                display_name="".join(instruction.operator.ascii_symbols) + "H",
            )

        # Add the new gate to the adjoint circuit. Note the order of operations here:
        # (AB)^H = B^H A^H, where H is adjoint, thus we prepend new gates, rather than append.
        adjoint_circ = adjoint_gate.add(adjoint_circ)
    return adjoint_circ

In [3]:

# monkey patch get_unitary() and adjoint() to the Circuit class

# set up device: Local Schroedinger Simulator
device = LocalSimulator()

In [4]:
@circuit.subroutine(register=True)
def minus_R_B(qubit):
    """
    Function to apply a minus sign to |B>|0>. This is achieved by applying XZX to the ancilla qubit.

    Args:
        qubit: the ancilla qubit on which we apply XZX.
    """
    # instantiate circuit object
    circ = Circuit()

    # Apply sequence XZX to given qubit
    circ.x(qubit).z(qubit).x(qubit)

    return circ


# Helper function to apply rotation -R0
@circuit.subroutine(register=True)
def minus_R_zero(qubits, use_explicit_unitary=False):
    """
    Function to implement transformation: |0,0,...0> -> -|0,0,...0>, all others unchanged.

    Args:
        qubits: list of qubits on which to apply the gates
        use_explicit_unitary (default False): Flag to specify that we could instead implement
        the desired gate using a custom gate defined by the unitary diag(-1,1,...,1).
    """

    circ = Circuit()

    # If the use_explicit_matrix flag is True, we just apply the unitary defined by |0,0,...0> -> -|0,0,...0>
    if use_explicit_unitary:
        # Create the matrix diag(-1,1,1,...,1)
        unitary = np.eye(2 ** len(qubits))
        unitary[0][0] = -1
        # Add a gate defined by this matrix
        circ.unitary(matrix=unitary, targets=qubits)

    # Otherwise implement the unitary using ancilla qubits:
    else:
        # Flip all qubits. We now need to check if all qubits are |1>, rather than |0>.
        circ.x(qubits)

        # If we have only 1 qubit, we only need to apply XZX to that qubit to pick up a minus sign on |0>
        if len(qubits) < 2:
            circ.z(qubits)

        # For more qubits, we use Toffoli (or CCNOT) gates to check that all the qubits are in |1> (since we applied X)
        else:

            # Dynamically add ancilla qubits, starting on the next unused qubit in the circuit
            # TODO: if this subroutine is being applied to a subset of qubits in a circuit, these ancilla
            # registers might already be used. We could pass in circ as an argument and add ancillas outside of
            # circ.targets
            ancilla_start = max(qubits) + 1

            # Check that the first two register qubits are both 1's using a CCNOT on a new ancilla qubit.
            circ.ccnot(qubits[0], qubits[1], ancilla_start)

            # Now add a CCNOT from each of the next register qubits, comparing with the ancilla we just added.
            # Target on a new ancilla. If len(qubits) is 2, this does not execute.
            for ii, qubit in enumerate(qubits[2:]):
                circ.ccnot(qubit, ancilla_start + ii, ancilla_start + ii + 1)

            # Apply a Z gate to the last ancilla qubit to pick up a minus sign if all of the register qubits are |1>
            ancilla_end = ancilla_start + len(qubits[2:])
            circ.z(ancilla_end)

            # Now uncompute to disentangle the ancilla qubits by applying CCNOTs in the reverse order to above.
            for jj, qubit in enumerate(reversed(qubits[2:])):
                circ.ccnot(qubit, ancilla_end - jj - 1, ancilla_end - jj)

            # Finally undo the last CCNOT on the first two register qubits.
            circ.ccnot(qubits[0], qubits[1], ancilla_start)

        # Flip all qubits back
        circ.x(qubits)

    return circ


@circuit.subroutine(register=True)
def grover_iterator(A, flag_qubit, qubits=None, use_explicit_unitary=False):
    """
    Function to implement the Grover iterator Q=A R_0 A* R_B.

    Args:
        A: Circuit defining the unitary A
        flag_qubit: Specifies which of the qubits A acts on labels the good/bad subspace.
                    Must be an element of qubits (if passed) or A.qubits.
        qubits: list of qubits on which to apply the gates (including the flag_qubit).
                If qubits is different from A.qubits, A is applied to qubits instead.
        use_explicit_unitary: Flag to specify that we should implement R_0 using using a custom
                              gate defined by the unitary diag(-1,1,...,1). Default is False.
    """
    # If no qubits are passed, apply the gates to the targets of A
    if qubits is None:
        qubits = A.qubits
    else:
        # If qubits are passed, make sure it's the right number to remap from A.
        if len(qubits) != len(A.qubits):
            raise ValueError(
                "Number of desired target qubits differs from number of targets in A".format(
                    flag_qubit=repr(flag_qubit)
                )
            )

    # Verify that flag_qubit is one of the qubits on which A acts, or one of the user defined qubits
    if flag_qubit not in qubits:
        raise ValueError(
            "flag_qubit {flag_qubit} is not in targets of A".format(flag_qubit=repr(flag_qubit))
        )

    # Instantiate the circuit
    circ = Circuit()

    # Apply -R_B to the flag qubit
    circ.minus_R_B(flag_qubit)

    # Apply A^\dagger. Use target mapping if different qubits are specified
    circ.add_circuit(A.adjoint(), target=qubits)

    # Apply -R_0
    circ.minus_R_zero(qubits, use_explicit_unitary)

    # Apply A, mapping targets if desired.
    circ.add_circuit(A, target=qubits)

    return circ


@circuit.subroutine(register=True)
def qaa(A, flag_qubit, num_iterations, qubits=None, use_explicit_unitary=False):
    """
    Function to implement the Quantum Amplitude Amplification Q^m, where Q=A R_0 A* R_B, m=num_iterations.

    Args:
        A: Circuit defining the unitary A
        flag_qubit: Specifies which of the qubits A acts on labels the good/bad subspace.
                    Must be an element of qubits (if passed) or A.qubits.
        num_iterations: number of applications of the Grover iterator Q.
        qubits: list of qubits on which to apply the gates (including the flag_qubit).
                If qubits is different from A.qubits, A is applied to qubits instead.
        use_explicit_unitary: Flag to specify that we should implement R_0 using using a custom
                              gate defined by the unitary diag(-1,1,...,1). Default is False.
    """
    # Instantiate the circuit
    circ = Circuit()

    # Apply the Grover iterator num_iterations times:
    for _ in range(num_iterations):
        circ.grover_iterator(A, flag_qubit, qubits, use_explicit_unitary)

    return circ

In [5]:
# helper function to apply XZX to given qubit
@circuit.subroutine(register=True)
def minus_R_B(qubit):
    """
    Function to apply a minus sign to |B>|0>. This goal is achieved by applying XZX to the ancilla qubit.

    Args:
        qubit: the ancilla qubit on which we apply XZX.
    """
    # instantiate circuit object
    circ = Circuit()

    # Apply sequence XZX to given qubit
    circ.x(qubit).z(qubit).x(qubit)

    return circ

In [6]:
# Helper function to apply rotation -R0
@circuit.subroutine(register=True)
def minus_R_zero(qubits, use_explicit_unitary=False):
    """
    Function to implement transformation: |0,0,...0> -> -|0,0,...0>, all others unchanged. 

    Args:
        qubits: list of qubits on which to apply the gates
        use_explicit_unitary (default False): Flag to specify that we could instead implement
        the desired gate using a custom gate defined by the unitary diag(-1,1,...,1).
    """

    circ = Circuit()

    # If the use_explicit_matrix flag is True, we just apply the unitary defined by |0,0,...0> -> -|0,0,...0>
    if use_explicit_unitary:
        # Create the matrix diag(-1,1,1,...,1)
        unitary = np.eye(2**len(qubits))
        unitary[0][0]=-1
        # Add a gate defined by this matrix
        circ.unitary(matrix=unitary, targets=qubits)

    # Otherwise implement the unitary using ancilla qubits:
    else:
        # Flip all qubits. We now must check whether all qubits are |1>, rather than |0>.
        circ.x(qubits)

        # If we have only 1 qubit, we only must apply XZX to that qubit to pick up a minus sign on |0>
        if len(qubits) < 2:
            circ.z(qubits)

        # For more qubits, we use Toffoli (or CCNOT) gates to verify the qubits are in |1> (after applying X)
        else:

            # Dynamically add ancilla qubits, starting on the next unused qubit in the circuit
            # NOTE: if this subroutine is being applied to a subset of qubits in a circuit, these ancilla
            # registers may already be used. We could pass in circ as an argument and add ancillas outside of
            # circ.targets instead, if desired.
            ancilla_start = max(qubits) + 1

            # Check that the first two register qubits are both 1's using a CCNOT on a new ancilla qubit.
            circ.ccnot(qubits[0],qubits[1],ancilla_start)

            # Now add a CCNOT from each of the next register qubits, comparing with the ancilla we just added.
            # Target on a new ancilla. If len(qubits) is 2, this does not execute.
            for ii,qubit in enumerate(qubits[2:]):
                circ.ccnot(qubit,ancilla_start+ii, ancilla_start+ii+1)

            # A Z gate applied to the last ancilla qubit gives a minus sign if all register qubits are |1>
            ancilla_end = ancilla_start + len(qubits[2:])
            circ.z(ancilla_end)

            # Now uncompute to disentangle the ancilla qubits by applying CCNOTs in the reverse order to the previous.
            for jj,qubit in enumerate(reversed(qubits[2:])):
                circ.ccnot(qubit,ancilla_end-jj-1, ancilla_end-jj)

            # Finally undo the last CCNOT on the first two register qubits.
            circ.ccnot(qubits[0],qubits[1],ancilla_start)

        # Flip all qubits back
        circ.x(qubits)

    return circ

In [7]:
qubits = [0,1,2,3,4,5,6,7,8,9,10] #i=0
qubits = [0,1,3,4,5,6,7,8,9,10,2] #i=1
qubits = [0,1,4,5,6,7,8,9,10,2,3] #i=2
circ = Circuit()
circ1=Circuit()
circ1.minus_R_B(qubits)
print(circ1)
circ.minus_R_zero(qubits)
print(circ)

T   : |0|1|2|
             
q0  : -X-Z-X-
             
q1  : -X-Z-X-
             
q2  : -X-Z-X-
             
q3  : -X-Z-X-
             
q4  : -X-Z-X-
             
q5  : -X-Z-X-
             
q6  : -X-Z-X-
             
q7  : -X-Z-X-
             
q8  : -X-Z-X-
             
q9  : -X-Z-X-
             
q10 : -X-Z-X-

T   : |0|1|2|
T   : |0|1|2|3|4|5|6|7|8|9|10|11|12|13 |14|15 |16 |17 |18 |19 |20 |21 |22|
                                                                          
q0  : -X-C---------------------------------------------------------C---X--
         |                                                         |      
q1  : -X-C---------------------------------------------------------C---X--
         |                                                         |      
q2  : -X-|---------------C----------C---X--------------------------|------
         |               |          |                              |      
q3  : -X-|---------------|-C-----C--|-X------------------------

In [8]:
from braket.circuits.noise_model import (
    GateCriteria,
    NoiseModel,
    ObservableCriteria,
)
from braket.circuits import Circuit, Observable, Gate
from braket.circuits.noises import (
    BitFlip,
    Depolarizing,
    TwoQubitDepolarizing,
)
from braket.devices import LocalSimulator
import numpy as np
import math

def noise_model():
    rng = np.random.default_rng()
    m = NoiseModel()
    
    two_q_depo_mu = 1 - 0.9311
    two_q_depo_sigma = 0.005
    bf_mu = 1 - 0.99752
    bf_sigma = 0.0015
    one_q_depo_mu = 1 - 0.9981
    one_q_depo_sigma = 0.00017
    for qi in range(11):
        z_bf_prob = bf_mu + bf_sigma * rng.standard_normal()
        z_bf_prob = 0.0 if z_bf_prob < 0.0 else z_bf_prob
        
        bf_prob = bf_mu + bf_sigma * rng.standard_normal()
        bf_prob = 0.0 if bf_prob < 0.0 else bf_prob
        
        one_q_depo_prob = one_q_depo_mu + one_q_depo_sigma * rng.standard_normal()
        one_q_depo_prob = 0.0 if one_q_depo_prob < 0.0 else one_q_depo_prob
        
        m.add_noise(BitFlip(z_bf_prob), ObservableCriteria(observables=Observable.Z, qubits=qi))
        #m.add_noise(BitFlip(bf_prob), ObservableCriteria(qubits=qi))
        
        m.add_noise(Depolarizing(one_q_depo_prob), GateCriteria(qubits=qi))
        for qj in range(11):
            if not qj == qi:
                two_q_depo_prob = two_q_depo_mu + two_q_depo_sigma * rng.standard_normal()
                two_q_depo_prob = 0.0 if two_q_depo_prob < 0.0 else two_q_depo_prob
                
                m.add_noise(TwoQubitDepolarizing(two_q_depo_prob), GateCriteria(gates=[Gate.CNot, Gate.Swap, Gate.CPhaseShift], qubits=[qi, qj]))
    return m
#qubits = [0,1,2,3,4,5,6,7,8,9,10]
#circ = Circuit()
#circ.minus_R_zero(qubits)
#print(circ)
#nm = noise_model()
#c = nm.apply(circ)
#print(c)
n_ite = 9;
mat_seq = np.array([
    [0,1,2,3,4,5,6,7,8,9,10],
    [0,1,3,4,5,6,7,8,9,10,2],
    [0,1,4,5,6,7,8,9,10,2,3],
    [0,1,5,6,7,8,9,10,2,3,4],
    [0,1,6,7,8,9,10,2,3,4,5],
    [0,1,7,8,9,10,2,3,4,5,6],
    [0,1,8,9,10,2,3,4,5,6,7],
    [0,1,9,10,2,3,4,5,6,7,8],
    [0,1,10,2,3,4,5,6,7,8,9]
])
mat_depo_2d = []
for b in range (0,100):
    for ii in range (0,n_ite):
        qubits = mat_seq[ii][:]
        circ = Circuit()
        circ.minus_R_zero(qubits)
        #print(circ)

        # apply the noise model to the circuit 
        nm = noise_model()
        c = nm.apply(circ)
        a1 = str(c).replace(' ', '')
        a2 = a1.replace('\n', '')
        #print(a2)
        import re
        #pattern1 = r'q(.*?)\n'
        #matches = re.findall(pattern1, str(c))
        #print(matches)
        pattern2 = r'-DEPO\((.*?)\)-'
        matches2 = re.findall(pattern2, a2)
        matches2_filtered = matches2[4::2]
        mat_depo = np.array([[float(number) for number in row.split(' ')] for row in matches2_filtered])
        mat_depo1 = mat_depo
        if (b==0):
            mat_depo_2d.append(mat_depo1)
        else:
            for i in range (9):
                mat_depo_2d[ii][i]+=mat_depo[i]
        # examine the noisy circuit 
        #print(c)
        #print(type(c))
for j in range(9):
    for a in range(9):
        mat_depo_2d[j][a] = mat_depo_2d[j][a]/100.0
array_depo_2d = np.array(mat_depo_2d)
result_dict = []
pairing_mat = [[(2, 11), (3, 12), (4,13), (5,14), (6,15), (7,16), (8, 17), (9, 18), (10,19)], 
               [(3, 11), (4, 12), (5,13), (6,14), (7,15), (8,16), (9, 17), (10, 18), (2,19)],
               [(4, 11), (5, 12), (6,13), (7,14), (8,15), (9,16), (10, 17), (2, 18), (3,19)],
               [(5, 11), (6, 12), (7,13), (8,14), (9,15), (10,16), (2, 17), (3, 18), (4,19)],
               [(6, 11), (7, 12), (8,13), (9,14), (10,15), (2,16), (3, 17), (4, 18), (5,19)],
               [(7, 11), (8, 12), (9,13), (10,14), (2,15), (3,16), (4, 17), (5, 18), (6,19)],
               [(8, 11), (9, 12), (10,13), (2,14), (3,15), (4,16), (5, 17), (6, 18), (7,19)],
               [(9, 11), (10, 12), (2,13), (3,14), (4,15), (5,16), (6, 17), (7, 18), (8,19)],
               [(10, 11), (2, 12), (3,13), (4,14), (5,15), (6,16), (7, 17), (8, 18), (9,19)]]

for i in range(len(pairing_mat)):
    for j in range(len(pairing_mat[i])):
        result_dict.append([mat_depo_2d[i][j][0],pairing_mat[i][j][0],pairing_mat[i][j][1]])

print('\n')
def bubble_sort_descending(arr):
    n = len(arr)

    for i in range(n):
        # Flag to optimize the algorithm by checking if any swaps were made
        swapped = False

        # Last i elements are already in place, so no need to compare them
        for j in range(0, n-i-1):
            # Swap if the element found is smaller than the next element
            if arr[j][0] < arr[j+1][0]:
                arr[j], arr[j+1] = arr[j+1], arr[j]
                swapped = True

        # If no two elements were swapped in the inner loop, the list is already sorted
        if not swapped:
            break
bubble_sort_descending(result_dict)
result_dict1=[]
result_dict1=result_dict

Q = np.zeros((len(result_dict1),len(result_dict1)))
for i in range(len(result_dict1)):
    for j in range(len(result_dict1)):
        if i==j:
            Q[i][j]=-5*len(result_dict1)
        elif (result_dict1[i][1] == result_dict1[j][1] or  result_dict1[i][2] == result_dict1[j][2]) :
            Q[i][j] = 2000
        else:
            Q[i][j] = 10 - 5 * (1-result_dict1[i][0])*(1-result_dict1[j][0])




In [9]:
import numpy as np
from scipy.optimize import minimize

# Step 3: Define the objective function
def objective_function(x, Q):
    return x.dot(Q).dot(x)

# Step 4: Define constraints (if any)

# Step 5: Initial guess or bounds (optional)
initial_guess = np.zeros(len(Q))
bounds = [(0, 1)] * len(Q)  # Binary variables, so each variable is between 0 and 1

# Step 6: Use a solver to minimize the objective function
result = minimize(objective_function, initial_guess, args=(Q,), method='COBYLA', bounds=bounds)

# Step 7: Extract the optimal binary vector and the minimum energy
optimal_x = result.x
minimum_energy = result.fun

# Step 8: Print the results
print("Optimal Binary Vector:", optimal_x)
print("Minimum Energy:", minimum_energy)


Optimal Binary Vector: [1. 1. 1. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Minimum Energy: -3283.61908678385


In [10]:

print(type(optimal_x))
for x in range(len(optimal_x)):
    if optimal_x[x] == 1:
        print((result_dict1[x][1],result_dict1[x][2]))

<class 'numpy.ndarray'>
(10, 13)
(6, 18)
(4, 19)
(5, 12)
(2, 15)
(9, 16)
(8, 14)
(7, 17)
(3, 11)


In [11]:
#We then go back to our initial algorithm and start with bits [0,1] and the rest 
#of the order (for qubits 2-10) is in increasing order of their correspondent 
#qubit (11-19). In this scenario it will be [0,1,2,9,8,5,4,6,10,3,7]